In [1]:
# Als je een zip-bestand hebt geüpload, eerst uitpakken
# !unzip -q /kaggle/input/pytorch-metric-learning.zip -d /kaggle/working/

# Installeer de whl-bestanden
# !pip install /kaggle/input/pytorch-metric-learning
!pip install pytorch_metric_learning --no-index --find-links=file:///kaggle/input/pytorch-metric-learning


Looking in links: file:///kaggle/input/pytorch-metric-learning
Processing /kaggle/input/pytorch-metric-learning/pytorch_metric_learning-2.6.0-py3-none-any.whl


In [2]:
import os
import h5py
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from PIL import Image
from io import BytesIO
from tqdm import tqdm
import torch
from torch.cuda.amp import autocast, GradScaler
from pytorch_metric_learning import losses, miners
from pytorch_metric_learning.utils.accuracy_calculator import AccuracyCalculator
from pytorch_metric_learning.distances import LpDistance
from pytorch_metric_learning.miners import PairMarginMiner, TripletMarginMiner
from torch import nn, optim
from torchvision import transforms

# Load data

In [3]:
"""2024 ISIC Challenge primary prize scoring metric

Given a list of binary labels, an associated list of prediction 
scores ranging from [0,1], this function produces, as a single value, 
the partial area under the receiver operating characteristic (pAUC) 
above a given true positive rate (TPR).
https://en.wikipedia.org/wiki/Partial_Area_Under_the_ROC_Curve.

(c) 2024 Nicholas R Kurtansky, MSKCC
"""


class ParticipantVisibleError(Exception):
    pass


def score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str, min_tpr: float=0.80) -> float:
    '''
    2024 ISIC Challenge metric: pAUC
    
    Given a solution file and submission file, this function returns the
    the partial area under the receiver operating characteristic (pAUC) 
    above a given true positive rate (TPR) = 0.80.
    https://en.wikipedia.org/wiki/Partial_Area_Under_the_ROC_Curve.
    
    (c) 2024 Nicholas R Kurtansky, MSKCC

    Args:
        solution: ground truth pd.DataFrame of 1s and 0s
        submission: solution dataframe of predictions of scores ranging [0, 1]

    Returns:
        Float value range [0, max_fpr]
    '''

    del solution[row_id_column_name]
    del submission[row_id_column_name]

    # check submission is numeric
    if not pd.api.types.is_numeric_dtype(submission.values):
        raise ParticipantVisibleError('Submission target column must be numeric')

    # rescale the target. set 0s to 1s and 1s to 0s (since sklearn only has max_fpr)
    v_gt = abs(np.asarray(solution.values)-1)
    
    # flip the submissions to their compliments
    v_pred = -1.0*np.asarray(submission.values)

    max_fpr = abs(1-min_tpr)

    # using sklearn.metric functions: (1) roc_curve and (2) auc
    fpr, tpr, _ = roc_curve(v_gt, v_pred, sample_weight=None)
    if max_fpr is None or max_fpr == 1:
        return auc(fpr, tpr)
    if max_fpr <= 0 or max_fpr > 1:
        raise ValueError("Expected min_tpr in range [0, 1), got: %r" % min_tpr)
        
    # Add a single point at max_fpr by linear interpolation
    stop = np.searchsorted(fpr, max_fpr, "right")
    x_interp = [fpr[stop - 1], fpr[stop]]
    y_interp = [tpr[stop - 1], tpr[stop]]
    tpr = np.append(tpr[:stop], np.interp(max_fpr, x_interp, y_interp))
    fpr = np.append(fpr[:stop], max_fpr)
    partial_auc = auc(fpr, tpr)

    #     # Equivalent code that uses sklearn's roc_auc_score
    #     v_gt = abs(np.asarray(solution.values)-1)
    #     v_pred = np.array([1.0 - x for x in submission.values])
    #     max_fpr = abs(1-min_tpr)
    #     partial_auc_scaled = roc_auc_score(v_gt, v_pred, max_fpr=max_fpr)
    #     # change scale from [0.5, 1.0] to [0.5 * max_fpr**2, max_fpr]
    #     # https://math.stackexchange.com/questions/914823/shift-numbers-into-a-different-range
    #     partial_auc = 0.5 * max_fpr**2 + (max_fpr - 0.5 * max_fpr**2) / (1.0 - 0.5) * (partial_auc_scaled - 0.5)
    
    return(partial_auc)

In [4]:
# # Load data
data_path = '/kaggle/input/isic-2024-challenge/'
path_train_hdf5 = os.path.join(data_path, 'train-image.hdf5')
path_test_hdf5 = os.path.join(data_path, 'test-image.hdf5')

path_train_meta = data_path + 'train-metadata.csv'
path_test_meta = data_path + 'test-metadata.csv'

import pandas as pd

# Bestaande data
data_path = '/kaggle/input/isic-2024-challenge/'
path_train_hdf5 = os.path.join(data_path, 'train-image.hdf5')
train_hdf5 = h5py.File(path_train_hdf5, 'r')
train_meta = pd.read_csv(os.path.join(data_path, 'train-metadata.csv'))


extra_dir_1 = '/kaggle/input/old-data/archive(4)/train-image/image/'
extra_meta_1 = pd.read_csv('/kaggle/input/old-data/archive(4)/train-metadata.csv')

extra_dir_2 = '/kaggle/input/old-data/archive(2)/train-image/image/'
extra_meta_2 = pd.read_csv('/kaggle/input/old-data/archive(5)/train-metadata.csv')

extra_dir_3 = '/kaggle/input/old-data/archive(3)/train-image/image/'
extra_meta_3 = pd.read_csv('/kaggle/input/old-data/archive(3)/train-metadata.csv')

# Combineer metadata (verondersteld dat image_id de kolomnaam is)
combined_meta = pd.concat([train_meta, extra_meta_1, extra_meta_2, extra_meta_3], ignore_index=True)
jpeg_dirs = [extra_dir_1, extra_dir_2, extra_dir_3]


/tmp/ipykernel_24/386316080.py:15: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  train_meta = pd.read_csv(os.path.join(data_path, 'train-metadata.csv'))


In [5]:
train_hdf5 = h5py.File(path_train_hdf5, 'r')
test_hdf5 = h5py.File(path_test_hdf5, 'r')

In [6]:
train_meta = pd.read_csv(path_train_meta)
test_meta = pd.read_csv(path_test_meta)

/tmp/ipykernel_24/938936479.py:1: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  train_meta = pd.read_csv(path_train_meta)


In [7]:
# read in the isic ids and target values
train_isic_ids = train_meta['isic_id'].values
train_isic_ids = train_meta[train_meta['lesion_id'].notnull()]['isic_id'].values

test_isic_ids = test_meta['isic_id'].values

train_targets = train_meta[train_meta['lesion_id'].notnull()]['target'].values

In [8]:
total_size = len(train_targets)
indices = np.arange(total_size)

train_size = int(total_size * 0.8)
val_size = total_size - train_size

train_indices, val_indices = train_test_split(indices, test_size=val_size, train_size=train_size)


In [9]:
num_samples = len(train_isic_ids)

In [10]:
import numpy as np
from torch.utils.data import WeightedRandomSampler

# Veronderstel dat je al de targets hebt (bijv. train_targets of labels)
class_counts = np.bincount(train_targets)  # Aantal samples per klasse
class_weights = 1. / class_counts  # Omgekeerde van het aantal samples per klasse
sample_weights = class_weights[train_targets]  # Gewichten per sample gebaseerd op hun klasse

# Maak een WeightedRandomSampler met deze sample gewichten
sampler = WeightedRandomSampler(weights=sample_weights, num_samples=num_samples, replacement=True)


# Pytorch gedeelte

In [11]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import ViTModel, ViTImageProcessor 
from torch import nn, optim
from PIL import Image
import numpy as np
from io import BytesIO

import random
class TripletISICDataset(Dataset):
    def __init__(self, hdf5_file=None, isic_ids=None, targets=None, jpeg_dirs=None, feature_extractor=None, transform=None):
        self.hdf5_file = hdf5_file
        self.isic_ids = isic_ids
        self.targets = targets
        self.jpeg_dirs = jpeg_dirs if jpeg_dirs is not None else []
        self.feature_extractor = feature_extractor
        self.transform = transform

    def __len__(self):
        return len(self.isic_ids)

    def __getitem__(self, idx):
        isic_id = str(self.isic_ids[idx])

        # Load anchor image
        anchor_image, anchor_label = self._load_image_and_label(idx)

        # Get positive and negative ids based on implemented logic
        positive_idx = self._get_positive_id(idx, anchor_label)
        negative_idx = self._get_negative_id(idx, anchor_label)

        # Load positive and negative images
        positive_image, _ = self._load_image_and_label(positive_idx)
        negative_image, _ = self._load_image_and_label(negative_idx)

        return (anchor_image, positive_image, negative_image), anchor_label

    def _get_positive_id(self, idx, anchor_label):
        positive_indices = np.where(self.targets == anchor_label)[0]
        positive_indices = positive_indices[positive_indices != idx]  # Vermijd dezelfde index
        if len(positive_indices) == 0:
            raise ValueError("Geen positieve sample gevonden")
        return np.random.choice(positive_indices)

    def _get_negative_id(self, idx, anchor_label):
        negative_indices = np.where(self.targets != anchor_label)[0]
        if len(negative_indices) == 0:
            raise ValueError("Geen negatieve sample gevonden")
        return np.random.choice(negative_indices)

    def _load_image_and_label(self, idx):
        isic_id = str(self.isic_ids[idx])
        image = None

        for jpeg_dir in self.jpeg_dirs:
            image_path = os.path.join(jpeg_dir, f"{isic_id}.jpeg")
            if os.path.exists(image_path):
                image = Image.open(image_path)
                break

        if image is None and self.hdf5_file:
            image = Image.open(BytesIO(self.hdf5_file[isic_id][()]))

        if image is None:
            raise FileNotFoundError(f"Afbeelding {isic_id} niet gevonden.")

        # Converteer de afbeelding naar RGB indien nodig
        if isinstance(image, np.ndarray):
            image = Image.fromarray(image)

        image = image.convert("RGB")

        # Toepassen van de transformaties
        if self.feature_extractor:
            image = self.feature_extractor(image)  # Zorg ervoor dat de extractor geen keyword 'images=' vereist
        elif self.transform:
            image = self.transform(image)
        else:
            image = torch.tensor(np.array(image)).permute(2, 0, 1).float() / 255.0

        label = self.targets[idx]
        return image, label

    def to_pytorch_loader(self, batch_size=32, shuffle=True, num_workers=4, sampler=None):
        if sampler:
            return DataLoader(self, batch_size=batch_size, sampler=sampler, num_workers=num_workers)
        else:
            return DataLoader(self, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)

    @staticmethod
    def collate_fn(batch):
        (anchor_images, positive_images, negative_images), labels = zip(*batch)
        anchor_images = torch.stack(anchor_images)
        positive_images = torch.stack(positive_images)
        negative_images = torch.stack(negative_images)
        labels = torch.tensor(labels)
        return (anchor_images, positive_images, negative_images), labels

# Stel de image transforms in voor ResNet-50
feature_extractor = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

2024-08-29 19:21:23.150705: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-29 19:21:23.150819: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-29 19:21:23.281478: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [12]:
import torch
import torch.nn as nn
from torchvision import models, transforms

# Laad het ResNet-50 model
model = models.resnet50(pretrained=False)  # Zet pretrained=False omdat je eigen gewichten laadt
# Pad naar de opgeslagen gewichten
weights_path = "/kaggle/input/resnet/pytorch/default/1/resnet50-0676ba61.pth"

# Laad de gewichten
model.load_state_dict(torch.load(weights_path, map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu')))

# model = models.resnet50("/kaggle/input/resnet/pytorch/default/1", pretrained=True)

# Laad het ResNet-50 model en vervang de laatste volledig verbonden laag
model.fc = nn.Linear(model.fc.in_features, 128)  # Pas de output aan naar 128 dimensies


# Voor training met triplet loss, kan het helpen om de output te normaliseren
class ResNet50TripletModel(nn.Module):
    def __init__(self, backbone):
        super(ResNet50TripletModel, self).__init__()
        self.backbone = backbone

    def forward(self, x):
        # Gebruik de backbone voor feature extractie
        x = self.backbone(x)
        # Normaliseer de embeddings
        x = nn.functional.normalize(x, p=2, dim=1)
        return x

# Maak een instance van het aangepaste ResNet50 model
triplet_model = ResNet50TripletModel(model)

# Definieer de image transforms (feature extractor voor ResNet-50)
# Stel de image transforms in voor ResNet-50
feature_extractor = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


# Verplaats het model naar het juiste device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
triplet_model = triplet_model.to(device)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [13]:
class COMTripletLoss(nn.Module):
    def __init__(self):
        super(COMTripletLoss, self).__init__()

    def forward(self, anchor, positive, negative):
        # Bepaal het centrum van de positieve klasse als het gemiddelde van de anker en positieve embeddings
        center = (anchor + positive) / 2
        
        # Bereken de afstand van anchor en positive tot het centrum
        pos_dist = (anchor - center).pow(2).sum(1) + (positive - center).pow(2).sum(1)
        
        # Bereken de afstand van negative tot het centrum
        neg_dist = (negative - center).pow(2).sum(1)
        
        # De loss is de som van de positieve afstanden min de negatieve afstand
        loss = pos_dist - neg_dist
        
        # Loss moet altijd positief zijn, daarom nemen we de gemiddelde waarde en nemen we de maximale waarde tussen loss en 0.
        return loss.clamp(min=0).mean()

In [14]:
optimizer = optim.AdamW(model.parameters(), lr=1e-5)
# Loss functie en miner voor semi-hard negatieve mijnbouw
# loss_func = losses.TripletMarginLoss(margin=0.2)

# Loss functie en miner voor semi-hard negatieve mijnbouw
loss_func = COMTripletLoss()

miner = TripletMarginMiner(margin=0.05, type_of_triplets="semihard")

from torch.optim.lr_scheduler import ReduceLROnPlateau
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True, min_lr=1e-7)

In [15]:
# Maak een instance van de COM-Triplet Loss
com_triplet_loss = COMTripletLoss()

def train_com_triplet_model(model, dataloader, num_epochs=10, optimizer=None, loss_func=None, miner=None, scheduler=None, save_path="model_weights.pth"):
        model.train()

        for epoch in range(num_epochs):
            epoch_loss = 0.0
            progress_bar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch")

            for batch in progress_bar:
                optimizer.zero_grad()

                # Unpack the triplets
                (anchor_images, positive_images, negative_images), labels = batch

                # Send to device
                anchor_images = anchor_images.to(device)
                positive_images = positive_images.to(device)
                negative_images = negative_images.to(device)

                # Compute embeddings
                anchor_embedding = model(anchor_images)
                positive_embedding = model(positive_images)
                negative_embedding = model(negative_images)

                embeddings = torch.cat([anchor_embedding, positive_embedding, negative_embedding], dim=0)
                labels = torch.cat([labels, labels, labels])

                # Mine hard negatives
                hard_pairs = miner(embeddings, labels)

                # Check if hard_pairs are empty
                if any(t.numel() == 0 for t in hard_pairs):
                    # Use a fallback loss calculation when no hard pairs are found
                    loss = loss_func(anchor_embedding, positive_embedding, negative_embedding)
                else:
                    # Extract the indices for hard pairs
                    anchor_idx, positive_idx, negative_idx = hard_pairs[:3]

                    # Select only the hard pairs
                    hard_anchor = embeddings[anchor_idx]
                    hard_positive = embeddings[positive_idx]
                    hard_negative = embeddings[negative_idx]

                    # Compute loss with only hard pairs
                    loss = loss_func(hard_anchor, hard_positive, hard_negative)

                loss.backward()
                optimizer.step()

                epoch_loss += loss.item()
                progress_bar.set_postfix({"loss": loss.item()})

            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss / len(dataloader):.4f}')

        if save_path:
            torch.save(model.state_dict(), save_path)
            print(f"Model gewichten opgeslagen in {save_path}")
        
# Creëer de dataset en dataloader
triplet_dataset = TripletISICDataset(
    hdf5_file=train_hdf5,
    isic_ids=combined_meta[combined_meta['lesion_id'].notnull()]['isic_id'].values,
    targets=combined_meta[combined_meta['lesion_id'].notnull()]['target'].values,
    jpeg_dirs=jpeg_dirs,
    feature_extractor=feature_extractor
)
train_dataloader = triplet_dataset.to_pytorch_loader(batch_size=16, shuffle=True, num_workers=2)

# Start het training proces
# train_com_triplet_model(model, train_dataloader, num_epochs=1, optimizer=optimizer, loss_func=loss_func, miner=miner, scheduler=scheduler)


In [16]:
import torch
import torch.nn as nn
import torchvision.models as models

# Laad het ResNet-50 model
model = models.resnet50(pretrained=False)

# Pas de laatste volledig verbonden laag aan voor het aantal outputfeatures (bijvoorbeeld 128)
# Dit is belangrijk voor triplet loss waarbij de outputdimensie moet passen
model.fc = nn.Linear(in_features=model.fc.in_features, out_features=128)

# Laad de getrainde gewichten
model.load_state_dict(torch.load("/kaggle/input/triplet_loss_resnet/pytorch/default/1/model_weights.pth"))
model.eval()  # Zet het model in evaluatiemodus

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [17]:
def custom_collate_fn(batch):
    images, ids = zip(*batch)  # Unzip de batch in images en ids
    images = [img.unsqueeze(0) for img in images]  # Voeg een batch dim toe aan elke image
    images = torch.cat(images, dim=0)  # Combineer alle images in één batch tensor
    return images, ids

In [18]:
import torchvision.transforms as transforms

# Definieer de transform om alle afbeeldingen naar 224x224 te schalen en naar tensor om te zetten
image_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Schaal afbeeldingen naar 224x224 pixels
    transforms.ToTensor(),          # Converteer naar een torch tensor
])

# Aangepaste datasetclass voor de testset
class TestISICDataset(Dataset):
    def __init__(self, hdf5_file, isic_ids, transform=None):
        self.hdf5_file = hdf5_file
        self.isic_ids = isic_ids
        self.transform = transform

    def __len__(self):
        return len(self.isic_ids)

    def __getitem__(self, idx):
        isic_id = str(self.isic_ids[idx])
        image = Image.open(BytesIO(self.hdf5_file[isic_id][()]))
        image = image.convert("RGB")
        if self.transform:
            image = self.transform(image)
        else:
            image = torch.tensor(np.array(image)).permute(2, 0, 1).float() / 255.0
        return image, isic_id

# Laad de testset in een DataLoader met de aangepaste collate_fn en transform
test_dataset = TestISICDataset(hdf5_file=test_hdf5, isic_ids=test_isic_ids, transform=image_transform)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2, collate_fn=custom_collate_fn)


In [19]:
def compute_embeddings(model, dataloader, device):
    model.eval()
    embeddings = []
    ids = []

    with torch.no_grad():
        for batch in dataloader:
            images, isic_ids = batch  # Zorg ervoor dat de batch correct opgesplitst wordt
            
            # Controleer of images een lijst is en stack alleen als dat nodig is
            if isinstance(images, list):
                images = torch.stack(images)  # Combineer de lijst van tensors tot één tensor
            
            images = images.to(device)  # Verplaats de tensor naar het apparaat
            
            # Bereken de embeddings met het model
            embedding = model(images)
            embeddings.append(embedding)
            ids.extend(isic_ids)  # Voeg IDs toe aan de lijst
    
    embeddings = torch.cat(embeddings)  # Combineer alle embeddings tot één tensor
    return embeddings, ids

# Laad het getrainde model en zet het op het juiste device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Bereken embeddings voor de testset
test_embeddings, test_ids = compute_embeddings(model, test_dataloader, device)


/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


In [20]:
# Placeholder predictiefunctie
def predict_labels(embeddings):
    # Dummy implementatie - vervang met je daadwerkelijke predictielogica
    return torch.zeros(len(embeddings))  # Vervang dit met daadwerkelijke voorspellingen

# Voer voorspellingen uit op de test embeddings
predicted_labels = predict_labels(test_embeddings)

# Maak de submissie aan in een pandas DataFrame
submission_df = pd.DataFrame({
    'isic_id': test_ids,
    'predicted_label': predicted_labels.cpu().numpy()  # Converteer tensor naar numpy array
})

# Opslaan van de submissie
submission_df.to_csv('submission.csv', index=False)
